In [3]:
import torch
from torch import nn, Tensor
# import torch.nn as nn
import torch.nn.functional as F


class Conv2dReLU(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        _layer = [] # 층의 개수가 달라도 호환되도록
        for _ in range(num_layers):
            _layer += [
                       nn.Conv2d(in_channels, out_channels, kernel_size=3),
                       nn.ReLU
            ] # append 안씀 -> configs=cfgs
        super().__init__(*_layers)

class VGG16(nn.Module):
    def __init__(self, in_channel=3):
        super(VGG16, self).__init__() # python 2.x -> python 3.x super().__init__()
        self.conv1 = Conv2dReLU(3, 64, kernel_size=3, padding=1)

        self.conv2 = Conv2dReLU(64, 64, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # self.pool = nn.MaxPool2d(2, 2)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) 
        # nn.AdaptiveAvgPool2d: 어떤 input_size가 들어와도 7*7로 output 고정 -> fc오류방지
        # nn.AdaptiveAvgPool2d는 fc input_size를 조정하므로 conv층이 끝나고 마지막에 넣기
        self.maxpool = nn.MaxPool2d(2, 2)

        self.fc6 = nn.Linear(in_features=512*7*7, out_features=4096)
        self.fc6 = nn.Linear(in_features=4096, out_features=4096)
        self.fc6 = nn.Linear(in_features=4096, out_features=1000)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        x = F.relu(self.conv3_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5_1(x))
        x = F.relu(self.conv5_2(x))
        x = F.relu(self.conv5_3(x))
        x = self.avgpool(x)

        x = x.view(-1, 7*7*512) # reshape

        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))

        return x

if __name__ == '__main__':
    from torchsummary import summary

    model = VGG16()
    print(summary(model, input_data=(3, 224, 224), verbose=0))

    from torchvision.models import vgg16

    model = vgg16(pretrained=False)
    print(summary(model, input_data(3, 224, 224), verbose=0))